Note: all ".to(device)" are deleted because Apple M1 chip does does support it.

Would recommend using Google Colab instead because it will be much faster

Source of LSTM codes:
https://github.com/quantumiracle/Popular-RL-Algorithms/blob/master/sac_v2_lstm.py

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#env" data-toc-modified-id="env-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>env</a></span><ul class="toc-item"><li><span><a href="#d22:-no-noise;-fixed-wgain" data-toc-modified-id="d22:-no-noise;-fixed-wgain-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>d22: no noise; fixed wgain</a></span></li></ul></li><li><span><a href="#make-LSTM-agent" data-toc-modified-id="make-LSTM-agent-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>make LSTM agent</a></span></li><li><span><a href="#TRAIN" data-toc-modified-id="TRAIN-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>TRAIN</a></span></li><li><span><a href="#Test" data-toc-modified-id="Test-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Test</a></span><ul class="toc-item"><li><span><a href="#test-and-save-variables" data-toc-modified-id="test-and-save-variables-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>test and save variables</a></span></li></ul></li><li><span><a href="#Animation" data-toc-modified-id="Animation-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Animation</a></span></li></ul></div>

# Set up

## install packages

In [ ]:
from google.colab import drive # import drive from google colab
drive.mount("/content/drive") 
!pip install neo
!pip install matplotlib_scalebar
!pip install ffmpeg
!pip install stable_baselines3
!pip install optuna

## import packages

In [ ]:
%cd /content/drive/MyDrive/ff_repo/Multifirefly-Project
from multiff_analysis.functions.RL import collect_agent_data, LSTM_functions, env
from multiff_analysis.functions.data_wrangling import find_patterns
from multiff_analysis.functions.data_visualization import animation_func
from RL.LSTM_agent_params import *


import os
import torch
import numpy as np
import pickle
from gym import spaces, Env
import torch
import optuna
from numpy import pi
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from torch.linalg import vector_norm
from IPython.display import HTML
from functools import partial
from optuna.pruners import MedianPruner
from optuna.samplers import TPESampler
plt.rcParams["animation.html"] = "html5"
torch.set_printoptions(sci_mode=False)
np.set_printoptions(suppress=True)

device_idx = 0
device = torch.device("cuda:" + str(device_idx) if torch.cuda.is_available() else "cpu")
## if using Jupyter Notebook
# device = "mps" if torch.backends.mps.is_available() else "cpu"
log_dir = "RL_models/LSTM_stored_models/LSTM_Jan_2"
os.makedirs(log_dir, exist_ok=True)
PLAYER = "agent"

## make agent

In [ ]:
replay_buffer_size=100
replay_buffer = LSTM_functions.ReplayBufferLSTM2(replay_buffer_size)
sac_model=LSTM_functions.SAC_Trainer(replay_buffer, state_space, action_space, hidden_dim=hidden_dim, action_range=action_range,\
                      gamma = gamma, soft_q_lr = soft_q_lr, policy_lr = policy_lr, alpha_lr = alpha_lr, \
                      batch_size = batch_size, update_itr = update_itr, reward_scale = reward_scale, \
                      target_entropy = target_entropy, soft_tau = soft_tau, train_freq = train_freq, device = device)

## load agent (optional)

In [ ]:
# Load agent
sac_model.load_model(log_dir)

# Load buffer
buffer_path = os.path.join(log_dir, 'buffer.pkl')
with open(buffer_path, 'rb') as f:
  replay_buffer = pickle.load(f)

# Train agent

In [ ]:
replay_buffer_size=100
replay_buffer = LSTM_functions.ReplayBufferLSTM2(replay_buffer_size)
sac_model=LSTM_functions.SAC_Trainer(replay_buffer, state_space, action_space, hidden_dim=hidden_dim, action_range=action_range,\
                      gamma = gamma, soft_q_lr = soft_q_lr, policy_lr = policy_lr, alpha_lr = alpha_lr, \
                      batch_size = batch_size, update_itr = update_itr, reward_scale = reward_scale, \
                      target_entropy = target_entropy, soft_tau = soft_tau, train_freq = train_freq, device = device)

env = env.AdaptForLSTM()
log_dir = "RL_models/LSTM_stored_models/LSTM_Jan_11"
os.makedirs(log_dir, exist_ok=True)

sac_model.soft_q_net1.train()
sac_model.soft_q_net2.train()
sac_model.policy_net.train()

sac_model, best_avg_reward_record = LSTM_functions.train_LSTM_agent(env, sac_model, device, action_dim=2, log_dir = log_dir,
                                                                    hidden_dim=128, train_freq=100, batch_size=10, update_itr=1,
                                                                    num_train_episodes=10000, eval_freq=15, max_steps=1024, AUTO_ENTROPY=True, 
                                                                    DETERMINISTIC=False, num_eval_episodes=3, print_episode_reward = True)

# Test agent

In [ ]:
num_epi_for_testing = 5
env = env.AdaptForLSTM()
env.flash_on_interval = 0.3
env.distance2center_cost = 0
sac_model.load_model(log_dir)
last_action = env.action_space.sample()            
hidden_out = (torch.zeros([1, 1, hidden_dim], dtype=torch.float).to(device), torch.zeros([1, 1, hidden_dim], dtype=torch.float).to(device))  
total_reward = 0
  
for eps in range(num_epi_for_testing):
    state =  env.reset()
    episode_reward = 0
    hidden_out = (torch.zeros([1, 1, hidden_dim], dtype=torch.float).to(device), torch.zeros([1, 1, hidden_dim], dtype=torch.float).to(device))  
    for step in range(max_steps):
        hidden_in = hidden_out
        action, hidden_out = sac_model.policy_net.get_action(state, last_action, hidden_in, deterministic = DETERMINISTIC)
        next_state, reward, done, _ = env.step(action)   
        last_action = action
        episode_reward += reward
        state=next_state
    total_reward += episode_reward
    print('Episode: ', eps, '| Episode Reward: ', episode_reward)
print("Average reward per episode: ", total_reward/num_epi_for_testing)

# Optuna (LSTM)

(my own codes)

##### sample_sac_params

In [ ]:
def sample_sac_params(trial):
    """
    Sampler for SAC hyperparams.
    :param trial: (optuna.trial)
    :return: (dict)
    """
  
    gamma = 1.0 - trial.suggest_float("1-gamma", 1e-4, 0.1, log=True)
    soft_q_lr = trial.suggest_float("soft_q_lr", 1e-5, 1, log=True)
    policy_lr = trial.suggest_float("policy_lr", 1e-5, 1, log=True)
    alpha_lr  = trial.suggest_float("alpha_lr", 1e-5, 1, log=True)
    batch_size  = trial.suggest_categorical('batch_size', [5, 10, 15, 20, 25, 30])
    update_itr = trial.suggest_categorical('update_itr', [1, 2, 3, 5])
    hidden_dim = trial.suggest_categorical('hidden_dim', [16, 32, 64, 100, 150, 200, 256])
    reward_scale = trial.suggest_categorical('reward_scale', [1, 3, 5, 10, 15, 20]) # I updated after running
    target_entropy = trial.suggest_categorical('target_entropy', [-1, -2, -3, -5, -8, -10]) # I updated after running
    soft_tau= trial.suggest_float("soft_tau", 1e-6, 1, log=True)
    #activation_fn


    return {
        'gamma': gamma,
        'soft_q_lr':soft_q_lr,
        'policy_lr':policy_lr,
        'alpha_lr':alpha_lr,
        'batch_size': batch_size,
        'update_itr':update_itr,
        'hidden_dim': hidden_dim,
        'reward_scale':reward_scale,
        'target_entropy':target_entropy,
        'soft_tau':soft_tau
    }

## put_in_fixed_params

In [ ]:
def put_in_fixed_params():
    return {
        'log_dir':  None, 
        'train_freq': 100, 
        'batch_size': 10, 
        'update_itr': 1,
        'num_train_episodes': 50, 
        'eval_freq': 10, 
        'max_steps': 1024, 
        'AUTO_ENTROPY': True, 
        'DETERMINISTIC': False, 
        'num_eval_episodes': 3, 
        'print_episode_reward':  True}

## objective

In [ ]:
def objective(trial: optuna.Trial) -> float: 
  try:
    # Sample hyperparameters
    kwargs = sample_sac_params(trial)
    gamma = kwargs['gamma']
    soft_q_lr = kwargs['soft_q_lr']
    policy_lr = kwargs['policy_lr']
    alpha_lr = kwargs['alpha_lr']
    batch_size = kwargs['batch_size']
    update_itr = kwargs['update_itr']
    hidden_dim = kwargs['hidden_dim']
    reward_scale = kwargs['reward_scale']
    target_entropy = kwargs['target_entropy']
    soft_tau = kwargs['soft_tau']


    kwargs = put_in_fixed_params()
    log_dir = kwargs['log_dir']
    train_freq = kwargs['train_freq']
    batch_size = kwargs['batch_size'] 
    update_itr = kwargs['update_itr']
    num_train_episodes = kwargs['num_train_episodes'] 
    eval_freq = kwargs['eval_freq']
    max_steps = kwargs['max_steps'] 
    AUTO_ENTROPY = kwargs['AUTO_ENTROPY'] 
    DETERMINISTIC = kwargs['DETERMINISTIC'] 
    num_eval_episodes = kwargs['num_eval_episodes'] 
    print_episode_reward = kwargs['print_episode_reward']


    env = env.AdaptForLSTM()
    action_space = env.action_space
    state_space = env.observation_space
    action_dim = action_space.shape[0]
    replay_buffer_size = 100
    replay_buffer = LSTM_functions.ReplayBufferLSTM2(replay_buffer_size)
    sac_model = LSTM_functions.SAC_Trainer(replay_buffer, state_space, action_space, hidden_dim=hidden_dim, action_range=action_range,\
                            gamma = gamma, soft_q_lr = soft_q_lr, policy_lr = policy_lr, alpha_lr = alpha_lr, \
                            batch_size = batch_size, update_itr = update_itr, reward_scale = reward_scale, \
                            target_entropy = target_entropy, soft_tau = soft_tau, train_freq = train_freq)
    sac_model.soft_q_net1.train()
    sac_model.soft_q_net2.train()
    sac_model.policy_net.train()

    sac_model, best_avg_reward_record = LSTM_functions.train_LSTM_agent(env, sac_model, device, log_dir = log_dir, action_dim=action_dim, 
                                                         hidden_dim=hidden_dim, train_freq=train_freq, batch_size=batch_size, update_itr=update_itr,
                                                         num_train_episodes=num_train_episodes, eval_freq=eval_freq, max_steps=max_steps, AUTO_ENTROPY=AUTO_ENTROPY, 
                                                         DETERMINISTIC=DETERMINISTIC, num_eval_episodes=num_eval_episodes, print_episode_reward=print_episode_reward)

  except ValueError as e:
    # Sometimes, random hyperparams can generate NaN
      print(e)

  return best_avg_reward_record

## run

In [ ]:
N_TRIALS = 100
N_STARTUP_TRIALS = 5

# Set pytorch num threads to 1 for faster training
torch.set_num_threads(1)
 
sampler = TPESampler(n_startup_trials=N_STARTUP_TRIALS)
## Do not prune before 1/3 of the max budget is used
# pruner = MedianPruner(n_startup_trials=N_STARTUP_TRIALS, n_warmup_steps=N_EVALUATIONS 2//3 3)

study = optuna.create_study(sampler=sampler, direction="maximize")
try:
    study.optimize(objective, n_trials=N_TRIALS)
except KeyboardInterrupt:
    pass

print("Number of finished trials: ", len(study.trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

print("  User attrs:")
for key, value in trial.user_attrs.items():
    print("    {}: {}".format(key, value))

# Animation

## collect data

In [ ]:
env = env.CollectInformationLSTM()
env.flash_on_interval = 0.3
env.distance2center_cost = 0
sac_model.load_model(log_dir)



monkey_information, ff_flash_sorted, ff_caught_T_sorted, ff_believed_position_sorted, \
           ff_real_position_sorted, ff_life_sorted, ff_flash_end_sorted, caught_ff_num, total_ff_num, \
           obs_ff_unique_identifiers, sorted_indices_all \
           = collect_agent_data.collect_agent_data_func(env, sac_model, n_steps = 2000, LSTM = True, hidden_dim= hidden_dim, device = device, deterministic = False)


ff_dataframe = make_ff_dataframe.make_ff_dataframe.make_ff_dataframe_func(monkey_information, ff_caught_T_sorted, ff_flash_sorted,  ff_real_position_sorted, ff_life_sorted, \
                                 player = "agent", max_distance = 400, data_folder_name = None, num_missed_index = 0, truncate = False, \
                                 obs_ff_unique_identifiers = obs_ff_unique_identifiers, sorted_indices_all = sorted_indices_all, \
                                 ff_noisy_xy_in_obs = ff_noisy_xy_in_obs)

min_point_index, max_point_index = np.min(np.array(ff_dataframe['point_index'])), np.max(np.array(ff_dataframe['point_index']))

n_ff_in_a_row = find_patterns.n_ff_in_a_row_func(ff_believed_position_sorted, distance_between_ff = 50)
visible_before_last_one_trials = find_patterns.visible_before_last_one_func(ff_dataframe)
disappear_latest_trials = find_patterns.disappear_latest_func(ff_dataframe)
ignore_sudden_flash_trials, ignore_sudden_flash_indices, ignore_sudden_flash_indices_for_anim, ff_target_pairs = find_patterns.ignore_sudden_flash_func(ff_dataframe, ff_real_position_sorted, max_point_index)
try_a_few_times_trials, try_a_few_times_indices, try_a_few_times_indices_for_anim = find_patterns.try_a_few_times_func(ff_caught_T_sorted, monkey_information, ff_believed_position_sorted, PLAYER, max_point_index)
give_up_after_trying_trials, give_up_after_trying_indices, give_up_after_trying_indices_for_anim = find_patterns.give_up_after_trying_func(ff_caught_T_sorted, monkey_information, ff_believed_position_sorted, PLAYER, max_point_index)

annotation_info = animation_func.make_annotation_info(caught_ff_num, max_point_index, n_ff_in_a_row, visible_before_last_one_trials, disappear_latest_trials, \
                                       ignore_sudden_flash_indices, give_up_after_trying_indices, try_a_few_times_indices)

## prepare for animation

In [ ]:
currentTrial = 2
num_trials = 7
k = 1
fig, ax = plt.subplots()
num_frames, anim_monkey_info, flash_on_ff_dict, alive_ff_dict, believed_ff_dict, new_num_trials, ff_dataframe_anim \
            = animation_func.prepare_for_animation(ff_dataframe, ff_caught_T_sorted, ff_life_sorted, ff_believed_position_sorted, 
            ff_real_position_sorted, ff_flash_sorted, monkey_information, k=k, currentTrial=currentTrial, num_trials=num_trials)
print("Number of frames is:", num_frames)

## make animation

In [ ]:
animate_func = partial(animation_func.animate, ax=ax, anim_monkey_info=anim_monkey_info, ff_dataframe_anim=ff_dataframe_anim, ff_real_position_sorted=ff_real_position_sorted, \
                         flash_on_ff_dict=flash_on_ff_dict, alive_ff_dict=alive_ff_dict, believed_ff_dict=believed_ff_dict, margin = 400)
anim = animation.FuncAnimation(fig, animate_func, frames=num_frames, interval=100, repeat=True) 
HTML(anim.to_html5_video())

## make animation with annotation

In [ ]:
annotation_info = animation_func.make_annotation_info(caught_ff_num+1, max_point_index, n_ff_in_a_row, visible_before_last_one_trials, disappear_latest_trials, \
                                        ignore_sudden_flash_indices, give_up_after_trying_indices, try_a_few_times_indices)
animate_annotated_func = partial(animation_func.animate_annotated, ax=ax, anim_monkey_info=anim_monkey_info, margin=margin, ff_dataframe=ff_dataframe, ff_real_position_sorted=ff_real_position_sorted, \
                                   flash_on_ff_dict=flash_on_ff_dict, alive_ff_dict=alive_ff_dict, believed_ff_dict=believed_ff_dict, ff_caught_T_sorted=ff_caught_T_sorted, annotation_info=annotation_info)
anim_annotated = animation.FuncAnimation(fig, animate_annotated_func, frames=num_frames, interval=100, repeat=True) 
HTML(anim_annotated.to_html5_video())

# Debug